In [1]:
pip install deeplake

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.3/580.3 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 98.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import numpy as np
import deeplake

In [3]:
ds = deeplake.load('hub://activeloop/fer2013-train')
trainx2,trainy2=ds.images,ds.labels
trainx2=np.array(trainx2)
trainy2=np.array(trainy2)
ds = deeplake.load('hub://activeloop/fer2013-public-test')
testx2,testy2=ds.images,ds.labels
testx2=np.array(testx2)
testy2=np.array(testy2)

-

Opening dataset in read-only mode as you don't have write permissions.


/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/fer2013-train



|

...

Opening dataset in read-only mode as you don't have write permissions.


|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/fer2013-public-test



/

hub://activeloop/fer2013-public-test loaded successfully.



In [4]:
trainx2=np.dstack([trainx2] * 3)
testx2=np.dstack([testx2]*3)
trainx2 = trainx2.reshape(-1, 48,48,3)
testx2= testx2.reshape (-1,48,48,3)
trainx2.shape

(28709, 48, 48, 3)

In [5]:
from keras.preprocessing.image import img_to_array, array_to_img

trainx2 = np.asarray([img_to_array(array_to_img(im, scale=False).resize((64,64))) for im in trainx2])
testx2 = np.asarray([img_to_array(array_to_img(im, scale=False).resize((64,64))) for im in testx2])

In [6]:
from tensorflow.keras.applications import ResNet50
input_layer=layers.Input(shape=(64,64,3))
model_ResNet50=ResNet50(weights='imagenet',input_tensor=input_layer,include_top=False)

model_ResNet50.summary()

94765736/94765736 [==============================] - 0s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 70, 70, 3)            0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 32, 32, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 32, 32, 64)           256       ['conv1_conv[0][0]']          
 on)                      

In [7]:
for layer in model_ResNet50.layers:
    layer.trainable = False
model = models.Sequential()
model.add(model_ResNet50)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(7, activation='softmax'))
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history2 = model.fit(trainx2, trainy2,batch_size=128, epochs=10,
                    validation_data=(testx2, testy2))

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


225/225 [==============================] - 25s 58ms/step - loss: 2.0830 - accuracy: 0.2773 - val_loss: 1.8832 - val_accuracy: 0.3263
Epoch 2/10
225/225 [==============================] - 10s 44ms/step - loss: 1.7263 - accuracy: 0.3452 - val_loss: 1.8663 - val_accuracy: 0.3338
Epoch 3/10
225/225 [==============================] - 10s 46ms/step - loss: 1.6473 - accuracy: 0.3703 - val_loss: 1.7021 - val_accuracy: 0.3555
Epoch 4/10
225/225 [==============================] - 10s 44ms/step - loss: 1.5982 - accuracy: 0.3854 - val_loss: 1.7258 - val_accuracy: 0.3586
Epoch 5/10
225/225 [==============================] - 10s 46ms/step - loss: 1.5551 - accuracy: 0.4039 - val_loss: 1.7414 - val_accuracy: 0.3408
Epoch 6/10
225/225 [==============================] - 10s 45ms/step - loss: 1.5338 - accuracy: 0.4131 - val_loss: 1.7170 - val_accuracy: 0.3653
Epoch 7/10
225/225 [==============================] - 10s 45ms/step - loss: 1.5092 - accuracy: 0.4220 - val_loss: 1.6836 - val_accuracy: 0.3703
Epo

In [8]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, AveragePooling2D, Flatten, Dense

In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history2 = model.fit(trainx2, trainy2,batch_size=128, epochs=10,
                    validation_data=(testx2, testy2))

Epoch 1/10
225/225 [==============================] - 16s 53ms/step - loss: 1.4679 - accuracy: 0.4397 - val_loss: 1.6816 - val_accuracy: 0.3792
Epoch 2/10
225/225 [==============================] - 10s 45ms/step - loss: 1.4555 - accuracy: 0.4468 - val_loss: 1.7466 - val_accuracy: 0.3572
Epoch 3/10
225/225 [==============================] - 10s 45ms/step - loss: 1.4610 - accuracy: 0.4426 - val_loss: 1.7276 - val_accuracy: 0.3678
Epoch 4/10
225/225 [==============================] - 11s 49ms/step - loss: 1.4450 - accuracy: 0.4507 - val_loss: 1.7012 - val_accuracy: 0.3764
Epoch 5/10
225/225 [==============================] - 11s 47ms/step - loss: 1.4283 - accuracy: 0.4540 - val_loss: 1.6725 - val_accuracy: 0.3809
Epoch 6/10
225/225 [==============================] - 10s 45ms/step - loss: 1.4229 - accuracy: 0.4555 - val_loss: 1.7468 - val_accuracy: 0.3639
Epoch 7/10
225/225 [==============================] - 10s 45ms/step - loss: 1.4234 - accuracy: 0.4557 - val_loss: 1.7364 - val_accuracy:

In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, AveragePooling2D, Flatten, Dense

def identity_block(X, filters):
    # Implementation of the identity block

    F1, F2, F3 = filters

    X_shortcut = X

    # First component of the main path
    X = Conv2D(F1, (1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)
    X = ReLU()(X)

    # Second component of the main path
    X = Conv2D(F2, (3, 3), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = ReLU()(X)

    # Third component of the main path
    X = Conv2D(F3, (1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)

    # Final step: Add shortcut to the main path
    X = Add()([X, X_shortcut])
    X = ReLU()(X)

    return X

def convolutional_block(X, filters, s=2):
    # Implementation of the convolutional block

    F1, F2, F3 = filters

    X_shortcut = X

    # First component of the main path
    X = Conv2D(F1, (1, 1), strides=(s, s))(X)
    X = BatchNormalization(axis=3)(X)
    X = ReLU()(X)

    # Second component of the main path
    X = Conv2D(F2, (3, 3), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = ReLU()(X)

    # Third component of the main path
    X = Conv2D(F3, (1, 1), strides=(1, 1))(X)
    X = BatchNormalization(axis=3)(X)

    # Shortcut path
    X_shortcut = Conv2D(F3, (1, 1), strides=(s, s))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut)

    # Final step: Add shortcut to the main path
    X = Add()([X, X_shortcut])
    X = ReLU()(X)

    return X

def ResNet50(input_shape=(64, 64, 3), classes=7):
    # Implementation of the ResNet-50 architecture

    X_input = Input(input_shape)

    # Zero-padding
    X = tf.keras.layers.ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2))(X)
    X = BatchNormalization(axis=3)(X)
    X = ReLU()(X)
    X = layers.MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, filters=[64, 64, 256], s=1)
    X = identity_block(X, filters=[64, 64, 256])
    X = identity_block(X, filters=[64, 64, 256])

    # Stage 3
    X = convolutional_block(X, filters=[128, 128, 512], s=2)
    X = identity_block(X, filters=[128, 128, 512])
    X = identity_block(X, filters=[128, 128, 512])
    X = identity_block(X, filters=[128, 128, 512])

    # Stage 4
    X = convolutional_block(X, filters=[256, 256, 1024], s=2)
    X = identity_block(X, filters=[256, 256, 1024])
    X = identity_block(X, filters=[256, 256, 1024])
    X = identity_block(X, filters=[256, 256, 1024])
    X = identity_block(X, filters=[256, 256, 1024])
    X = identity_block(X, filters=[256, 256, 1024])

    # Stage 5
    X = convolutional_block(X, filters=[512, 512, 2048], s=2)
    X = identity_block(X, filters=[512, 512, 2048])
    X = identity_block(X, filters=[512, 512, 2048])

    # Average Pooling
    X = AveragePooling2D((2, 2))(X)

    # Output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax')(X)

    # Create model
    model = tf.keras.models.Model(inputs=X_input, outputs=X)

    return model

# Create ResNet-50 model with specific input shape
model = ResNet50(input_shape=(64, 64, 3))

# Display model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 70, 70, 3)            0         ['input_2[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv2d (Conv2D)             (None, 32, 32, 64)           9472      ['zero_padding2d[0][0]']      
                                                                                                  
 batch_normalization (Batch  (None, 32, 32, 64)           256       ['conv2d[0][0]']          

In [11]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history2 = model.fit(trainx2, trainy2, epochs=10,
                    validation_data=(testx2, testy2))

Epoch 1/10
898/898 [==============================] - 89s 62ms/step - loss: 1.9048 - accuracy: 0.2872 - val_loss: 2.2616 - val_accuracy: 0.2956
Epoch 2/10
898/898 [==============================] - 53s 59ms/step - loss: 1.6697 - accuracy: 0.3764 - val_loss: 2.2522 - val_accuracy: 0.3199
Epoch 3/10
898/898 [==============================] - 53s 59ms/step - loss: 1.5869 - accuracy: 0.4036 - val_loss: 1.9450 - val_accuracy: 0.2948
Epoch 4/10
898/898 [==============================] - 53s 59ms/step - loss: 1.5088 - accuracy: 0.4305 - val_loss: 1.7248 - val_accuracy: 0.3221
Epoch 5/10
898/898 [==============================] - 54s 60ms/step - loss: 1.5563 - accuracy: 0.4137 - val_loss: 6.0575 - val_accuracy: 0.1532
Epoch 6/10
898/898 [==============================] - 53s 59ms/step - loss: 1.5656 - accuracy: 0.4169 - val_loss: 18.8892 - val_accuracy: 0.2901
Epoch 7/10
898/898 [==============================] - 54s 60ms/step - loss: 1.5295 - accuracy: 0.4209 - val_loss: 2.2240 - val_accuracy